<a href="https://colab.research.google.com/github/anascime/anascime/blob/output/Interface_Nao_Disruptiva_v07062023_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center> <font color="#acacac"> PD-00063-3078/2022 - Etapa E11 </center> </font>

---
# <center> **Simulador de Alternativas Metodológicas para Cálculo dos Indicadores Coletivos de Qualidade do Serviço de Energia Elétrica** </center>
---

## <center> <font color="#23AD40"> **Abordagem não-disruptiva** </center> </font>




![picture](https://drive.google.com/uc?export=download&id=1LaBLjc7rYNe-o2QvYLJRXlBtnk8BNjmz)

## Configurações para rodar o simulador

### Pré-Configurações

**LEIA-ME**

**Caminho de saída para Máquina Local e Google Colab:**

**Máquina Local:**
1. Siga as seguintes instruções: https://research.google.com/colaboratory/local-runtimes.html
2. Selecionar caminho em output_path.
3. Deixar contra-barra final no caminho (\\).

**Google Colab:**
1. Deixar output_path em branco.
2. O arquivo de saída será salvo no ambiente virtual do Colab e será necessário fazer o download.
3. Para fazer o download, acesse o símbolo da pasta, no canto esquerdo da tela, abaixo do símbolo {x}.
4. Posicionar o cursor do mouse no nome do arquivo gerado.
5. Clicar nos três pontos, no lado direito.
6. Selecionar "Fazer download".

In [ ]:
output_path = "" #@param {type:"string"}

### Importar bibliotecas


In [ ]:
#@title
!pip install scikit-fuzzy
!pip install wquantiles
!pip install kneed

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 994.0/994.0 kB 20.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-fuzzy: filename=scikit_fuzzy-0.4.2-py3-none-any.whl size=894073 sha256=875aa0876c62d649b8c15a203a0271f11f726d5b132a5cbb9ff4da276530a559
  Stored in directory: /root/.cache/pip/wheels/4f/86/1b/dfd97134a2c8313e519bcebd95d3fedc7be7944db022094bc8
Successfully built scikit-fuzzy
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#@title
import numpy as np # Operações matemáticas
import pandas as pd # Operações de base de dados
import matplotlib.pyplot as plt # Plotar gráficos
import seaborn as sns # Suplemento gráficos
from scipy import stats, spatial # Estatística
import statsmodels.api as sm # Estatística
from statsmodels.stats.outliers_influence import variance_inflation_factor # VIF
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif # VIF
from kneed import KneeLocator # Identificador de quebra estrutural do tipo Elbow

import pickle # Serializador de objetos - Salvar resultados
from tqdm.notebook import tqdm # Barra de progresso
import time # Operações de tempo

import skfuzzy  # Calcula cluster fuzzy
from sklearn.cluster import KMeans # Calcula cluster k-means
from sklearn.metrics import silhouette_score # Calcula o índice de silhueta
from scipy.stats import ks_2samp, zscore # Teste Kolmogorov-Smirnov de duas amostras, escore z de uma amostra
from sklearn.preprocessing import MinMaxScaler # Mínimo-Máximo
from wquantiles import quantile # Quantis

import plotly.express as px # Gráficos

import warnings # Alertas
warnings.filterwarnings("ignore")

current_time = time.strftime("%d%m%Y_%H%M")

### Importar módulos de cálculo

## Importar base de dados de conjuntos

In [ ]:
#@title
# Base de dados
data_url = r'https://drive.google.com/uc?export=download&id=1TABCMJ0rngb6BfkfUoylKGNlgcsHD3XG'

df = pd.read_excel(data_url, sheet_name = 'Base Final', na_values = 'NA')
dicio = pd.read_excel(data_url, sheet_name = 'Dicionário de dados', na_values = 'NA')

In [ ]:
socioeconomica = True #@param {type:"boolean"}
# Limpar dicionário

if socioeconomica:
  dicio_clean = dicio.loc[(~dicio['Dimensão'].isin(['Identificador', 'Indicador', 'Limite'])) &
                      (dicio['Pré-selecionado'] != 'Não') &
                      (dicio['Excluir Nulos'] != 'Sim')]
else:
  dicio_clean = dicio.loc[(~dicio['Dimensão'].isin(['Identificador', 'Indicador', 'Limite'])) &
                      (dicio['Pré-selecionado'] != 'Não') &
                      (dicio['Excluir Nulos'] != 'Sim') &
                      (dicio['Dimensão'] != 'Socioeconômica')]

# Variáveis candidatas
candvar = dicio_clean.loc[~dicio_clean['Dimensão'].isin(['Indicador Médio'])]

# Encontrar grupos e sinais
grupos = candvar.set_index('VARIAVEL')['Subdimensão'].to_dict()
sinais = candvar.set_index('VARIAVEL')['Sinal'].to_dict()
grupos_nome = candvar['Subdimensão'].unique()

## Motor de cálculo

### Seleção de atributos

In [ ]:
#@title
dec_y = 'DEC_MEDIO'
fec_y = 'FEC_MEDIO'

In [ ]:
#@markdown **Caso se tenha uma base de atributos pré-selecionadas, inserir nos campos abaixo separados por ponto-vírgula. Caso contrário, deixe os campos em branco.**

#@markdown * Selecione o modelo pré-calibrado
variaveis_pre = "GA" #@param ["GA", "SW"]

if variaveis_pre == 'GA':
  varlist_dec_pre = "COMP_M_ALIM;NUC_IND;AESP;PC_NUC_URB;ERMT_NU_1F;PLUV;IDHM;VIO;NUC_COM;PC_POST_MAD"
  varlist_fec_pre = "COMP_M_ALIM;PLUV;PC_NUC_URB;AESP;PC_NUC_COM;NUC_IND;TORRE_MAD;AVEG_MED_AREA;ERMT_NU_1F_AA"
else:
  varlist_dec_pre = "ENE_AA;ERMT_PROT;PLUV;PC_NUC_URB;PC_TRAFO_1F;AESP;PC_NUC_COM;VIO_HAB;PC_POST_MAD"
  varlist_fec_pre = "COMP_M_ALIM;AVEG_MED_ALT_AREA;NUC_COM;DIAS_CHUVA_AM;VIO;IDHM;ERMT_NU_1F;TORRE_MAD"

if varlist_dec_pre != "" and varlist_fec_pre != "":
  varlist_dec_pre = varlist_dec_pre.split(';')
  varlist_fec_pre = varlist_fec_pre.split(';')

#@markdown ---

#@markdown **Selecione abaixo os modelos que se deseja estimar:**

#@markdown * Rodar Stepwise?
run_stepwise = False #@param {type:"boolean"}
#@markdown * Rodar Algoritmo Genético?
run_ga = False #@param {type:"boolean"}

#@markdown ---

#@markdown * Considerar subdimensões?
var_sub = True #@param {type:"boolean"}
#@markdown * Considerar sinais?
var_sinais = True #@param {type:"boolean"}
#@markdown * Considerar BIC?
bic = True #@param {type:"boolean"}
#@markdown * Considerar AIC?
aic = True #@param {type:"boolean"}




In [ ]:
drop_missing = True #@param {type:"boolean"}
# Identificar variáveis com missing:
missings = dict(df.isna().mean())
find_missing = np.array(list(filter(lambda x: x[1] > 0, missings.items())))[:, 0]

if drop_missing:
  candvar = candvar.loc[~candvar['VARIAVEL'].isin(find_missing)]
  drop = find_missing
else:
  drop = []

In [ ]:
#@title Calcular correlações
verificar_par = True #@param {type:"boolean"}
verificar_rel = False #@param {type:"boolean"}


var_dicio = dicio_clean['VARIAVEL'].loc[(~dicio_clean['VARIAVEL'].isin(['DEC_MEDIO', 'FEC_MEDIO'])) & (~dicio_clean['VARIAVEL'].isin(drop))]

pearson = df[dicio_clean['VARIAVEL']].corr('pearson')
spearman = df[dicio_clean['VARIAVEL']].corr('spearman')

exclude_vars_dec = ['DEC_MEDIO', 'FEC_MEDIO', 'DEC_BT', 'DEC_MT', 'FEC_BT', 'FEC_MT']
exclude_vars_fec = ['DEC_MEDIO', 'FEC_MEDIO', 'DEC_BT', 'DEC_MT', 'FEC_BT', 'FEC_MT']

for i in var_dicio:
  if verificar_par:
    for j in var_dicio:
      if i != j and i != dec_y and j != dec_y and i != fec_y and j != fec_y:
        # Verificar se correlações superam 0,9
        if abs(pearson[i][j]) >= 0.9 and abs(spearman[i][j]) >= 0.9:

          #Excluir variável com menor correlação com DEC
          if abs(pearson[dec_y][i]) > abs(pearson[dec_y][j]):
            exclude_vars_dec.append(j)
          else:
            exclude_vars_dec.append(i)

          #Excluir variável com menor correlação com FEC
          if abs(pearson[fec_y][i]) > abs(pearson[fec_y][j]):
            exclude_vars_fec.append(j)
          else:
            exclude_vars_fec.append(i)

for i in var_dicio:
  if verificar_rel:
    if abs(pearson[fec_y][i]) < 0.2 or abs(spearman[fec_y][i]) < 0.2:
      exclude_vars_fec.append(i)

    if abs(pearson[dec_y][i]) < 0.2 or abs(spearman[dec_y][i]) < 0.2:
      exclude_vars_dec.append(i)

exclude_vars_dec = np.unique(exclude_vars_dec)
exclude_vars_fec = np.unique(exclude_vars_fec)

candvar_dec = list(set(candvar['VARIAVEL']).difference(set(exclude_vars_dec)))
candvar_fec = list(set(candvar['VARIAVEL']).difference(set(exclude_vars_fec)))

print('Número de variáveis candidatas para explicar DEC: ' + str(len(candvar_dec)))
print('Número de variáveis candidatas para explicar FEC: ' + str(len(candvar_fec)))

Número de variáveis candidatas para explicar DEC: 197
Número de variáveis candidatas para explicar FEC: 197


In [ ]:
#@title
# Pré-seleção por correlação:
subset_dec = np.log(df[[dec_y] + list(candvar_dec)] + 1).dropna()
subset_fec = np.log(df[[fec_y] + list(candvar_fec)] + 1).dropna()
subset_dec['COD_CONJ'] = df['COD_CONJ']
subset_fec['COD_CONJ'] = df['COD_CONJ']

In [ ]:
#@title
# Criar variáveis de grupos e sinais

if var_sub:
  dimensoes = dicio_clean[['VARIAVEL', 'Subdimensão']].loc[dicio_clean['VARIAVEL'].isin(var_dicio)]
  grupos_sw = {dim: list(dimensoes.loc[dimensoes['Subdimensão'] == dim]['VARIAVEL']) for dim in dimensoes['Subdimensão'].unique()}
else:
  grupos_sw = None

Motor de cálculo do Stepwise

In [ ]:
#@title
class Stepwise:
    def __init__(self, data, criteria = {}, constraints = {}, varlim = None, maxvarg = 1, signals = None, groups = None):
      self.data = data
      self.criteria = {'BIC': False, 'AIC': False, 'Signals': False, 'Groups': False}
      self.constraints = {'VIF': 5, 'pvalue': 0.01}
      self.history = {'R2-adj': [None], 'BIC': [None], 'AIC': [None], 'Varlist': [[]], 'Model': [None]}
      self.varlim = varlim
      self.maxvarg = maxvarg
      self.signals = signals
      self.groups = groups

      # Atualizar lista de critérios
      for k in self.criteria.keys():
        if k in list(criteria.keys()):
          self.criteria[k] = criteria[k]

      # Atualizar lista de restrições
      for k in self.constraints.keys():
        if k in list(constraints.keys()):
          self.constraints[k] = constraints[k]

      # Iniciar histórico de resultados:
      self.history = {'R2-adj': [None], 'BIC': [None], 'AIC': [None], 'Varlist': [[]], 'Model': [None]}

    def _setup(self):
      pass

    def _objfun(self, varlist, verif_vif = True):

      check_g = True
      check_varlim = True

      # Checar grupos
      if self.criteria['Groups']:
        assert self.groups != None

      # Chegar número de variáveis:
      if self.varlim != None:
        if len(varlist) > self.varlim:
          check_varlim = False

      if check_g and check_varlim:

        subset_df = self.data[[self.y] + varlist].dropna()

        ols = sm.OLS(endog = subset_df[self.y],
                    exog = sm.add_constant(subset_df[varlist]),
                    missing = 'drop')

        res = ols.fit()

        check_p = True
        check_vif = True
        check_g = True
        check_s = True

        # Checar p-valor
        check_p = all([p <= self.constraints['pvalue'] for p in res.pvalues])

        # Checar VIF
        if verif_vif:
          if self.constraints['VIF'] != None:
            if len(varlist) > 3:
              check_vif = all([variance_inflation_factor(np.array(np.array(sm.add_constant(subset_df[varlist]))), i) <= self.constraints['VIF'] for i in range(1, len(varlist) + 1)])

        # Checar grupos
        if self.criteria['Groups']:

          cont_grupos = {dim: np.sum([1 for v in varlist if v in self.groups[dim]]) for dim in self.groups.keys()}
          if any([v > self.maxvarg for v in list(cont_grupos.values())]):
            check_g = False

        # Checar sinais
        if self.criteria['Signals']:

          params = res.params
          for v in varlist:
            if params[v] < 0 and self.signals[v] == '+':
              check_s = False
            if params[v] > 0 and self.signals[v] == '-':
              check_s = False

        # Checar se atende a todos os critérios
        if all([check_p, check_vif, check_g, check_s]):
          return res.rsquared_adj, res.bic, res.aic, res.f_pvalue, res

        else:
          return None, None, None, None, None

    def _check_convergency(self):
      pass

    def fit(self, y, x, method = 'bidirectional'):
        self.x = x
        self.y = y
        self.method = method

        self.converge = False

        # Título da tabela de saída
        print("{:<5} | {:<17} | {:<17} | {:<11} | {:<8} | {:<8}".format('Etapa', 'Variável Inserida', 'Variável Excluída', 'R2 ajustado', ' BIC', ' AIC'))
        i = 1
        while self.converge == False:
          if len(self.history['Varlist'][-1]) == self.varlim:
            self.converge = True
          else:
            self.converge = self._step1(i)
          i += 1

    def _step1(self, i):

      # Pegar lista de variáveis já incluídas

      varlist = self.history['Varlist'][-1]

      # Adicionar novas variáveis

      ## Encontrar lista de variáveis candidatas ainda não selecionadas
      candvar = list(set(self.x).difference(set(varlist)))
      selected_var = None
      inserted = ''
      excluded = ''
      result = []

      for var in candvar:
        temp_varlist = varlist + [var]

        # Rodar e salvar resultados
        r2, bic, aic, pf, model = self._objfun(temp_varlist)

        result.append({'var': var,
                      'R2': r2,
                      'BIC': bic,
                      'AIC': aic,
                      'PF': pf})

      # Selecionar a melhor variável

      ## Ordenar resultados por R2
      result = list(filter(lambda x: x['R2'] != None, result))

      if len(result) == 0:
        return True

      ordered_result = list(sorted(result, key = lambda x: x['R2'], reverse = True))

      ## Verificar se melhor resultado é melhor que o resultado vigente

      if self.history['R2-adj'][-1] != None:

        # Checar R2 ajustado
        if self.history['R2-adj'][-1] <= ordered_result[0]['R2']:

          check_criteria = True

          if self.criteria['BIC']:
            # Avaliar BIC
            if self.history['BIC'][-1] <= ordered_result[0]['BIC']:
              check_criteria = False

          if self.criteria['AIC']:
            # Avaliar AIC
            if self.history['AIC'][-1] <= ordered_result[0]['AIC']:
              check_criteria = False

          if check_criteria:
            selected_var = ordered_result[0]['var']

      else:
        selected_var = ordered_result[0]['var']

      # Inserir variável selecionada na listagem de variáveis temporária
      if selected_var != None:

        varlist = self.history['Varlist'][-1] + [selected_var]

        r2, bic, aic, pf, model = self._objfun(varlist)
        self.history['R2-adj'].append(r2)
        self.history['BIC'].append(bic)
        self.history['AIC'].append(aic)
        self.history['Varlist'].append(varlist)
        self.history['Model'].append(model)

        inserted = selected_var

        #print("{:<5} | {:<17} | {:<17} | {:<11} | {:<8} | {:<8}".format(i, inserted, excluded, round(self.history['R2-adj'][-1], 4), round(self.history['BIC'][-1], 2), round(self.history['AIC'][-1], 2)))

        # Se o método for birecional
        if self.method == 'bidirectional' and len(self.history['Varlist'][-1]) > 3:
          ## Encontrar lista de variáveis candidatas ainda não selecionadas
          candvar = self.history['Varlist'][-1]
          selected_varlist = None
          result = []

          for var in candvar:

            temp_varlist = list(set(candvar).difference(set([var])))

            # Rodar e salvar resultados
            r2, bic, aic, pf, model = self._objfun(temp_varlist)

            result.append({'Var': var,
                          'R2': r2,
                          'BIC': bic,
                          'AIC': aic,
                          'PF': pf,
                          'varlist': temp_varlist})

          # Selecionar a melhor variável

          ## Ordenar resultados por R2
          result = list(filter(lambda x: x['R2'] != None, result))
          ordered_result = list(sorted(result, key = lambda x: x['R2'], reverse = True))

          ## Verificar se melhor resultado é melhor que o resultado vigente

          if self.history['R2-adj'][-1] != None:

            # Checar R2 ajustado
            if self.history['R2-adj'][-1] <= ordered_result[0]['R2']:

              check_criteria = True

              if self.criteria['BIC']:
                # Avaliar BIC
                if self.history['BIC'][-1] <= ordered_result[0]['BIC']:
                  check_criteria = False

              if self.criteria['AIC']:
                # Avaliar AIC
                if self.history['AIC'][-1] <= ordered_result[0]['AIC']:
                  check_criteria = False

              if check_criteria:
                selected_varlist = ordered_result[0]['varlist']

              if selected_varlist != None:

                #varlist = self.history['Varlist'][-1] + [selected_var]
                r2, bic, aic, pf, model = self._objfun(selected_varlist)

                self.history['R2-adj'].append(r2)
                self.history['BIC'].append(bic)
                self.history['AIC'].append(aic)
                self.history['Varlist'].append(selected_varlist)
                self.history['Model'].append(model)

                excluded = selected_var

        print("{:<5} | {:<17} | {:<17} | {:<11} | {:<8} | {:<8}".format(i, inserted, excluded, round(self.history['R2-adj'][-1], 4), round(self.history['BIC'][-1], 2), round(self.history['AIC'][-1], 2)))
        return False

      else:
        return True

In [ ]:
#@title

varlist_dec_sw = []
varlist_fec_sw = []

if run_stepwise:
  print('Seleção Stepwise DEC')
  sw_dec = Stepwise(data = subset_dec,
                    criteria = {'BIC': bic, 'AIC': aic, 'Groups': var_sub, 'Signals': var_sinais},
                    varlim = None,
                    groups = grupos_sw,
                    signals = sinais)
  sw_dec.fit(y = dec_y, x = candvar_dec)

  print('\nSeleção Stepwise FEC')
  sw_fec = Stepwise(data = subset_fec,
                    criteria = {'BIC': bic, 'AIC': aic, 'Groups': var_sub, 'Signals': var_sinais},
                    varlim = None,
                    groups = grupos_sw,
                    signals = sinais)

  sw_fec.fit(y = fec_y, x = candvar_fec)

  varlist_dec_sw = sw_dec.history['Varlist'][-1]
  varlist_fec_sw = sw_fec.history['Varlist'][-1]

  sw_dec_r2 = {i: j for i, j in enumerate(sw_dec.history['R2-adj']) if j is not None}
  sw_fec_r2 = {i: j for i, j in enumerate(sw_fec.history['R2-adj']) if j is not None}

  # Melhores Modelos
  print('Modelo DEC')
  print(sw_dec.history['Model'][-1].summary())

  print('Modelo FEC')
  print(sw_fec.history['Model'][-1].summary())

In [ ]:
#@title
#Preparar tabela para exportação dos resultados

if run_stepwise:
  r2_res = {'const': 0}
  bic_res = {'const': 0}
  aic_res = {'const': 0}

  for vnum in range(len(varlist_dec_sw)):
    vlist = varlist_dec_sw[:vnum+1]
    y = subset_dec[dec_y]
    x = sm.add_constant(subset_dec[vlist])
    res = sm.OLS(y, x, hasconst = True).fit()
    r2_res[varlist_dec_sw[vnum]] = res.rsquared_adj
    bic_res[varlist_dec_sw[vnum]] = res.bic
    aic_res[varlist_dec_sw[vnum]] = res.aic

  mod_sw_dec = sm.OLS(subset_dec[dec_y], sm.add_constant(subset_dec[varlist_dec_sw]), hasconst = True).fit()

  res_sw_dec = pd.DataFrame({'Coeficientes': mod_sw_dec.params,
                            'p-valor': mod_sw_dec.pvalues,
                            'R2 ajustado': r2_res,
                            'BIC': bic_res,
                            'AIC': aic_res})

  r2_res = {'const': 0}
  bic_res = {'const': 0}
  aic_res = {'const': 0}

  for vnum in range(len(varlist_fec_sw)):
    vlist = varlist_fec_sw[:vnum+1]
    y = subset_fec[fec_y]
    x = sm.add_constant(subset_fec[vlist])
    res = sm.OLS(y, x, hasconst = True).fit()
    r2_res[varlist_fec_sw[vnum]] = res.rsquared_adj
    bic_res[varlist_fec_sw[vnum]] = res.bic
    aic_res[varlist_fec_sw[vnum]] = res.aic

  mod_sw_fec = sm.OLS(subset_fec[fec_y], sm.add_constant(subset_fec[varlist_fec_sw]), hasconst = True).fit()

  res_sw_fec = pd.DataFrame({'Coeficientes': mod_sw_fec.params,
                            'p-valor': mod_sw_fec.pvalues,
                            'R2 ajustado': r2_res,
                            'BIC': bic_res,
                            'AIC': aic_res})

Motor de Cálculo do Algoritmo Genético

In [ ]:
#@title
class Agent:
    def __init__(self, varlist, parent_key = None):
        self.id = id(self)
        self.lenvar = len(varlist)

        if parent_key == None:
            self._gen_new_key()
        else:
            self.key = parent_key

        self.varlist = self._get_attributes(varlist)

        self.result = {'Model': None,
                       'R2': None,
                       'BIC': None,
                       'AIC': None,
                       'VIF': None,
                       'Error': False}

    def _gen_new_key(self, min_threshold = 0.9):
        # Essa função gera uma chave aleatória
        self.key = []
        threshold = np.random.uniform(min_threshold, 1.0)
        varkeys = np.random.uniform(0, 1, size = self.lenvar + 1)
        self.key.append(threshold)
        self.key.extend(varkeys)

    def _decode_key(self, varlist):
        # Essa função gera um dicionário que decodifica a chave
        varlist_dict =  {}

        for i, var in enumerate(varlist):
            varlist_dict[var] = self.key[1 + i]

        return varlist_dict

    def _get_attributes(self, varlist):
        # Essa função retorna as variáveis que superam o threshold
        my_varlist = self._decode_key(varlist)
        return dict(list(filter(lambda v: v[1] >= self.key[0], my_varlist.items())))

    def fit(self,
            data,
            yvar,
            groups = None,
            group_names = None,
            signals = None,
            max_vif = None,
            varlims = (None, None),
            max_vargroup = 1,
            debug = False):

        # Inicializar checagens
        check_lims = True
        check_groups = True
        check_signals = True
        check_pvalue = True
        check_vif = True

        # Checar limites inferior e superior do modelo
        if varlims[0] != None:
            if len(self.varlist) < varlims[0]:
                check_lims = False

        if varlims[1] != None:
            if len(self.varlist) > varlims[1]:
                check_lims = False

        if check_lims:

            # Checar grupos se groups != None
            if groups != None:
                count_groups = {gname: len(list(filter(lambda v: groups[v] == gname, self.varlist.keys()))) for gname in group_names}
                check_groups = all([v <= max_vargroup for v in count_groups.values()])

            if check_groups:
                # Rodar modelo
                y = data[yvar]
                x = sm.add_constant(data[self.varlist])
                model = sm.OLS(endog = y,
                               exog = x,
                               hasconst = True,
                               missing = 'drop')

                result = model.fit()



                # Checar sinais

                params = result.params

                if signals != None:
                    for var, par in params.items():
                        if var != 'const':
                            if signals[var] == '-' and par > 0:
                                check_signals = False
                                break
                            if signals[var] == '+' and par < 0:
                                check_signals = False
                                break

                if check_signals:

                    pvalues = result.pvalues

                    # Checar significância

                    check_pvalues = all([p <= 0.01 for p in pvalues.values])

                    if check_pvalues:

                        if max_vif != None:
                            # Checar VIF
                            check_vif = all([vif(x, i) <= max_vif for i in range(1, x.shape[1])])


                        if check_vif:
                            self.result['Model'] = result
                            self.result['R2'] = result.rsquared_adj
                            self.result['AIC'] = result.aic
                            self.result['BIC'] = result.bic


        self.result['Error'] = {'Limites': check_lims,
                                'Grupos': check_groups,
                                'Sinais': check_signals,
                                'pvalor': check_pvalue,
                                'vif': check_vif}


        if debug == True:
            if check_lims == False:
                print('Fora dos limites de variável: {} variáveis no modelo'.format(len(self.varlist)))

            if check_groups == False:
                print('Muitas variáveis por grupo')

            if check_signals == False:
                print('Algum coeficiente viola o sinal esperado')

            if check_pvalue == False:
                print('Algum coeficiente não é significativo')

            if check_vif == False:
                print('Há evidências de multicolinearidade')

class GenAlg:
    def __init__(self, n_pop, data, yvar, varlist):
        self.data = data
        self.yvar = yvar
        self.varlist = varlist
        self.generation = 0

        # Iniciar população
        self.population = []

        for n in tqdm(range(n_pop), desc = 'Iniciando população'):
            self.population.append(Agent(varlist))

        self.results = []

    def fit(self,
            n_sims = 99,
            groups = None,
            group_names = None,
            varlims = (None, None),
            signals = None,
            max_vif = 5,
            max_vargroup = 1,
            elite_share = 0.3,
            mutant_share = 0.3,
            elite_bias = 0.7):

        self.elite_share = elite_share
        self.mutant_share = mutant_share
        self.elite_bias = elite_bias

        for gen in tqdm(range(n_sims), desc = 'Rodando gerações...'):

            for ag in self.population:
                if ag.result['Model'] == None:
                    ag.fit(self.data,
                           self.yvar,
                           groups = groups,
                           group_names = group_names,
                           signals = signals,
                           varlims = varlims,
                           max_vif = max_vif,
                           max_vargroup = max_vargroup)

            self._rank()

            if gen % 10 == 0:
                print('Melhor desempenho da geração {}: {}'.format(gen, np.max([ag.result['R2'] if ag.result['R2'] != None else 0 for ag in self.population])))

            if gen <  n_sims:
                self._transition()

    def _rank(self):

        r2s = {ag: ag.result['R2'] for ag in self.population}
        r2s = np.array(sorted(r2s.items(), key = lambda a: a[1] if a[1] != None else 0, reverse = True))
        self.population = r2s[:, 0]

        self.results.append({'Generation': self.generation,
                            'Model': self.population[0].result['Model'],
                            'R2': self.population[0].result['R2'],
                            'BIC': self.population[0].result['BIC'],
                            'AIC': self.population[0].result['AIC']})

    def _transition(self):

        elite_pop = int(self.elite_share * len(self.population))
        mutant_pop = int(self.mutant_share * len(self.population))
        nonelite_pop = len(self.population) - elite_pop - mutant_pop

        elite = list(self.population[:elite_pop].copy())

        mutant = []

        for n in range(mutant_pop):
            mutant.append(Agent(self.varlist))

        children = []

        nonelite = self.population[elite_pop:].copy()

        for n in range(nonelite_pop):
            elite_parent = np.random.choice(elite)
            nonelite_parent = np.random.choice(nonelite)
            key_child = self._crossover(elite_parent, nonelite_parent)
            children.append(Agent(self.varlist, key_child))

        self.population = elite + mutant + children



    def _crossover(self, elite, nonelite):

        key_elite = elite.key
        key_nonelite = nonelite.key

        key_child = []

        for k1, k2 in zip(key_elite, key_nonelite):
            key_child.append(np.random.choice([k1, k2], p = (self.elite_bias, 1-self.elite_bias)))

        return key_child

In [ ]:
#@markdown Definir número de gerações:
nsims = 9999 #@param {type:"integer"}
#@markdown Definir população:
npop = 250 #@param {type:"integer"}

varlist_dec_ga = []
varlist_fec_ga = []

if run_ga:

  GA_DEC = GenAlg(n_pop = npop,
                data = subset_dec,
                yvar = dec_y,
                varlist = candvar_dec)

  GA_DEC.fit(n_sims = nsims,
          groups = grupos,
          group_names = grupos_nome,
          signals = sinais,
          varlims = (1, 30))

  GA_FEC = GenAlg(n_pop = npop,
                data = subset_fec,
                yvar = fec_y,
                varlist = candvar_fec)

  GA_FEC.fit(n_sims = nsims,
          groups = grupos,
          group_names = grupos_nome,
          signals = sinais,
          varlims = (1, 30))

  varlist_dec_ga = list(GA_DEC.results[-1]['Model'].params.keys())
  varlist_fec_ga = list(GA_FEC.results[-1]['Model'].params.keys())

In [ ]:
#@title

if run_ga:
  ga_ordered_varlist_dec = []

  while len(ga_ordered_varlist_dec) < len(varlist_dec_ga) - 1:
    test_varlist = list(set(varlist_dec_ga[1:]).difference(set(ga_ordered_varlist_dec)))
    r2_models = {}
    for v in test_varlist:
      r2_models[v] = sm.OLS(subset_dec[dec_y], sm.add_constant(subset_dec[ga_ordered_varlist_dec + [v]]), hasconst = True).fit().rsquared_adj

    r2_models = dict(sorted(r2_models.items(), key = lambda x: x[1], reverse = True))
    ga_ordered_varlist_dec.append(list(r2_models.keys())[0])

  varlist_dec_ga = ga_ordered_varlist_dec

  ga_ordered_varlist_fec = []

  while len(ga_ordered_varlist_fec) < len(varlist_fec_ga) - 1:
    test_varlist = list(set(varlist_fec_ga[1:]).difference(set(ga_ordered_varlist_fec)))
    r2_models = {}
    for v in test_varlist:
      r2_models[v] = sm.OLS(subset_fec[fec_y], sm.add_constant(subset_fec[ga_ordered_varlist_fec + [v]]), hasconst = True).fit().rsquared_adj

    r2_models = dict(sorted(r2_models.items(), key = lambda x: x[1], reverse = True))
    ga_ordered_varlist_fec.append(list(r2_models.keys())[0])

  varlist_fec_ga = ga_ordered_varlist_fec

In [ ]:
#@title
if run_ga:
  r2_res = {'const': 0}
  bic_res = {'const': 0}
  aic_res = {'const': 0}

  for vnum in range(len(varlist_dec_ga)):
    vlist = varlist_dec_ga[:vnum+1]
    y = subset_dec[dec_y]
    x = sm.add_constant(subset_dec[vlist])
    res = sm.OLS(y, x, hasconst = True).fit()
    r2_res[varlist_dec_ga[vnum]] = res.rsquared_adj
    bic_res[varlist_dec_ga[vnum]] = res.bic
    aic_res[varlist_dec_ga[vnum]] = res.aic

  mod_ga_dec = sm.OLS(subset_dec[dec_y], sm.add_constant(subset_dec[varlist_dec_ga]), hasconst = True).fit()

  res_ga_dec = pd.DataFrame({'Coeficientes': mod_ga_dec.params,
                            'p-valor': mod_ga_dec.pvalues,
                            'R2 ajustado': r2_res,
                            'BIC': bic_res,
                            'AIC': aic_res})

  r2_res = {'const': 0}
  bic_res = {'const': 0}
  aic_res = {'const': 0}

  for vnum in range(len(varlist_fec_ga)):
    vlist = varlist_fec_ga[:vnum+1]
    y = subset_fec[fec_y]
    x = sm.add_constant(subset_fec[vlist])
    res = sm.OLS(y, x, hasconst = True).fit()
    r2_res[varlist_fec_ga[vnum]] = res.rsquared_adj
    bic_res[varlist_fec_ga[vnum]] = res.bic
    aic_res[varlist_fec_ga[vnum]] = res.aic

  mod_ga_fec = sm.OLS(subset_fec[fec_y], sm.add_constant(subset_fec[varlist_fec_ga]), hasconst = True).fit()

  res_ga_fec = pd.DataFrame({'Coeficientes': mod_ga_fec.params,
                            'p-valor': mod_ga_fec.pvalues,
                            'R2 ajustado': r2_res,
                            'BIC': bic_res,
                            'AIC': aic_res})

In [ ]:
#@title Restrição do número de variáveis

if run_stepwise:
  varlist_dec_sw = sw_dec.history['Varlist'][-1]
  varlist_fec_sw = sw_fec.history['Varlist'][-1]
  sw_dec_r2 = {i: j for i, j in enumerate(sw_dec.history['R2-adj']) if j is not None}
  sw_fec_r2 = {i: j for i, j in enumerate(sw_fec.history['R2-adj']) if j is not None}

if run_ga:
  varlist_dec_ga = list(GA_DEC.results[-1]['Model'].params.keys())[1:]
  varlist_fec_ga = list(GA_FEC.results[-1]['Model'].params.keys())[1:]
  ga_dec_r2 = dict(enumerate(res_ga_dec['R2 ajustado'].values))
  ga_fec_r2 = dict(enumerate(res_ga_fec['R2 ajustado'].values))

# Escolha entre dois métodos de restrição de modelo
rest_var = "R2" #@param ["Nao restringir", "R2", "Quebra"]

def rest_R2(var_r2):
  diff = [list(var_r2.values())[i] - list(var_r2.values())[i - 1] for i in range(1, len(var_r2.values()))]

  rest_num = len(var_r2)
  for i, r2 in enumerate(diff):
    if r2 <= 0.01:
      rest_num = list(var_r2.keys())[i]
      break
  return rest_num

def rest_quebra(var_r2, S = 1):
  kneedle = KneeLocator(range(len(var_r2)), list(var_r2.values()), S=S, curve="concave", direction="increasing")
  return list(var_r2.keys())[kneedle.knee]

if rest_var == 'R2':
  if run_stepwise:
    var_rest_dec_sw = rest_R2(sw_dec_r2)
    var_rest_fec_sw = rest_R2(sw_fec_r2)

  if run_ga:
    var_rest_dec_ga = rest_R2(ga_dec_r2)
    var_rest_fec_ga = rest_R2(ga_fec_r2)

if rest_var == 'Quebra':
  #Utilizando o algoritmo desenvolvido por  Satopaa et al (2011)
  if run_stepwise:
    var_rest_dec_sw = rest_quebra(sw_dec_r2)
    var_rest_fec_sw = rest_quebra(sw_fec_r2)

  if run_ga:
    var_rest_dec_ga = rest_quebra(ga_dec_r2)
    var_rest_fec_ga = rest_quebra(ga_fec_r2)

if run_stepwise:
  print('DEC SW', varlist_dec_sw[:var_rest_dec_sw])
  print('FEC SW', varlist_fec_sw[:var_rest_fec_sw])
  varlist_dec_sw_r = varlist_dec_sw[:var_rest_dec_sw]
  varlist_fec_sw_r = varlist_fec_sw[:var_rest_fec_sw]

if run_ga:
  print('DEC GA', res_ga_dec[:var_rest_dec_ga])
  print('FEC GA', varlist_fec_ga[:var_rest_fec_ga])
  varlist_dec_ga_r = list(res_ga_dec.index[1:])[:var_rest_dec_ga]
  varlist_fec_ga_r = list(res_ga_fec.index[1:])[:var_rest_fec_ga]

### Salvar resultados

In [ ]:
#@markdown **Tabela-resumo dos modelos**


vars = []

for i in range(40):
  vars.append({})

  try:
    vars[-1]['Pré/DEC'] = varlist_dec_pre[i]
  except:
    vars[-1]['Pré/DEC'] = ""

  try:
    vars[-1]['Pré/FEC'] = varlist_fec_pre[i]
  except:
    vars[-1]['Pré/FEC'] = ""

  if run_stepwise:
    try:
      vars[-1]['SW/DEC'] = varlist_dec_sw[i]
    except:
      vars[-1]['SW/DEC'] = ""

    try:
      vars[-1]['SW/FEC'] = varlist_fec_sw[i]
    except:
      vars[-1]['SW/FEC'] = ""

    try:
      vars[-1]['SW/DEC Rest'] = varlist_dec_sw_r[i]
    except:
      vars[-1]['SW/DEC Rest'] = ""

    try:
      vars[-1]['SW/FEC Rest'] = varlist_fec_sw_r[i]
    except:
      vars[-1]['SW/FEC Rest'] = ""

  if run_ga:
    try:
      vars[-1]['GA/DEC'] = varlist_dec_ga[i]
    except:
      vars[-1]['GA/DEC'] = ""

    try:
      vars[-1]['GA/FEC'] = varlist_fec_ga[i]
    except:
      vars[-1]['GA/FEC'] = ""

    try:
      vars[-1]['GA/DEC Rest'] = varlist_dec_ga_r[i]
    except:
      vars[-1]['GA/DEC Rest'] = ""

    try:
      vars[-1]['GA/FEC Rest'] = varlist_fec_ga_r[i]
    except:
      vars[-1]['GA/FEC Rest'] = ""


vars = pd.DataFrame.from_dict(vars)

with pd.ExcelWriter(rf'{output_path}Resumo_Modelos_{current_time}.xlsx') as writer:

    if run_stepwise:
      res_sw_dec.to_excel(writer, sheet_name='Stepwise DEC')
      res_sw_fec.to_excel(writer, sheet_name='Stepwise FEC')
    if run_ga:
      res_ga_dec.to_excel(writer, sheet_name='GA DEC')
      res_ga_fec.to_excel(writer, sheet_name='GA FEC')

    vars.to_excel(writer, sheet_name='Resumo')


In [ ]:
#@title

if run_ga:
  with open(rf'{output_path}ga_dec_{current_time}.pickle', 'wb') as handle:
      pickle.dump(GA_DEC, handle, protocol=pickle.HIGHEST_PROTOCOL)

  with open(rf'{output_path}ga_fec_{current_time}.pickle', 'wb') as handle:
      pickle.dump(GA_FEC, handle, protocol=pickle.HIGHEST_PROTOCOL)

if run_stepwise:

  with open(rf'{output_path}sw_fec_{current_time}.pickle', 'wb') as handle:
      pickle.dump(sw_fec, handle, protocol=pickle.HIGHEST_PROTOCOL)

  with open(rf'{output_path}sw_dec_{current_time}.pickle', 'wb') as handle:
      pickle.dump(sw_dec, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
#@markdown **Selecione a lista de variaveis que será utilizada em diante**
model_select = "Pre-selecionada" #@param ["Pre-selecionada", "Stepwise", "GA"]

varlist_dec = None
varlist_fec = None

if model_select == "Pre-selecionada":
  varlist_dec = varlist_dec_pre
  varlist_fec = varlist_fec_pre

elif model_select == "Stepwise":
  varlist_dec = varlist_dec_sw_r
  varlist_fec = varlist_fec_sw_r

elif model_select == 'GA':
  varlist_dec = varlist_dec_ga_r
  varlist_fec = varlist_fec_ga_r

#@markdown ---

print('Atributos selecionados para DEC: ')
print(varlist_dec)

print('Atributos selecionados para FEC: ')
print(varlist_fec)

Atributos selecionados para DEC: 
['COMP_M_ALIM', 'NUC_IND', 'AESP', 'PC_NUC_URB', 'ERMT_NU_1F', 'PLUV', 'IDHM', 'VIO', 'NUC_COM', 'PC_POST_MAD']
Atributos selecionados para FEC: 
['COMP_M_ALIM', 'PLUV', 'PC_NUC_URB', 'AESP', 'PC_NUC_COM', 'NUC_IND', 'TORRE_MAD', 'AVEG_MED_AREA', 'ERMT_NU_1F_AA']


In [ ]:
#@markdown ---
#@markdown A partir daqui, escolha o caminho a seguir:
caminho = "Vigente (ANEEL)" #@param ["Vigente (ANEEL)", "Fuzzy"]

#@markdown ---


### Caminhos alternativos

#### Abordagem vigente - ANEEL

In [ ]:
#@title
class Cluster_Dinam:
    def __init__(self, data, varlist_dec, varlist_fec, id_var, sinais, minconj = 50, maxconj = 100, maxdp = 3, maxhetero = 20):
        self.data = data
        self.varlist_dec = varlist_dec
        self.varlist_fec = varlist_fec
        self.id_var = id_var
        self.sinais = sinais
        self.minconj = minconj
        self.maxconj = maxconj
        self.maxdp = maxdp
        self.maxhetero = maxhetero

        self.id_dict = {id: self.data.iloc[id][self.id_var] for id in range(len(self.data))}
        self.conj_dict = {self.data.iloc[id][self.id_var]: id for id in range(len(self.data))}

    def fit(self):
        self.CalcDist()
        self.CalcHet()
        self.CalcCluster()
        #self.CalcANI()

    def CalcDist(self):
        self.distance_dec = spatial.distance.cdist(self.data[self.varlist_dec].to_numpy(), self.data[self.varlist_dec].to_numpy(), metric = 'euclidean')
        self.distance_fec = spatial.distance.cdist(self.data[self.varlist_fec].to_numpy(), self.data[self.varlist_fec].to_numpy(), metric = 'euclidean')

    def CalcHet(self):
        self.het_dec = 100 * self.distance_dec / (self.maxdp * np.sqrt(len(self.varlist_dec)))
        self.het_fec = 100 * self.distance_fec / (self.maxdp * np.sqrt(len(self.varlist_fec)))

    def _CalcCluster(self, het, ids, limiar = 20, min_elem = 50, max_elem = 100):

        clusters = {}

        for row in tqdm(range(len(het)), desc = 'Calculando Clusters', total = len(het)):

            filtered = np.array(list(filter(lambda x: x[1] <= limiar, enumerate(het[:, row]))))

            # Se não atinge o número mínimo de elementos
            if len(filtered) < min_elem:
                sort_values = np.array(list(sorted(enumerate(het[:, row]), key = lambda x: x[1])))
                cluster = sort_values[:int(min_elem)]

            # Se ultrapassa o número máximo de elementos
            elif len(filtered) > max_elem:
                sort_values = np.array(list(sorted(enumerate(het[:, row]), key = lambda x: x[1])))
                cluster = sort_values[:int(max_elem)]

            # Caso Contrário
            else:
                cluster = filtered

            cluster = [(ids[c[0]], c[1]) for c in cluster]
            clusters[ids[row]] = cluster

        return clusters

    def CalcCluster(self):

        self.clusters_dec = self._CalcCluster(self.het_dec,
                                             self.id_dict,
                                             self.maxhetero,
                                             self.minconj,
                                             self.maxconj)

        self.clusters_fec = self._CalcCluster(self.het_fec,
                                             self.id_dict,
                                             self.maxhetero,
                                             self.minconj,
                                             self.maxconj)

        self.data_cluster_dec = []
        self.data_cluster_fec = []

        for id1 in tqdm(self.conj_dict.keys(), desc = 'Consolidando dados de clusters'):

            for row in self.clusters_dec[id1]:
                dt = {}
                dt[self.id_var] = int(id1)
                dt['Conj_Sem'] = int(row[0])
                dt['Distancia'] = self.distance_dec[self.conj_dict[id1], self.conj_dict[row[0]]]
                dt['Heterogeneidade'] = row[1]

                self.data_cluster_dec.append(dt)

            for row in self.clusters_fec[id1]:
                dt = {}
                dt[self.id_var] = int(id1)
                dt['Conj_Sem'] = int(row[0])
                dt['Distancia'] = self.distance_fec[self.conj_dict[id1], self.conj_dict[row[0]]]
                dt['Heterogeneidade'] = row[1]

                self.data_cluster_fec.append(dt)

        self.data_cluster_dec = pd.DataFrame.from_dict(self.data_cluster_dec)
        self.data_cluster_fec = pd.DataFrame.from_dict(self.data_cluster_fec)

    def CalcANI(self, data):
        # Calcular parâmetro de outliers
        # Unifificar variáveis em uma
        vars = list(set(self.varlist_dec).union(set(self.varlist_fec)))

        mean = data[vars].mean().to_dict()
        std = data[vars].std().to_dict()
        desv = {v: mean[v] + 3 * std[v] for v in vars}

        # Calcular base sem outliers
        outlier_dec = data.copy()
        outlier_fec = data.copy()
        outlier = data.copy()

        for v in self.varlist_dec:
            outlier_dec = outlier_dec.loc[outlier_dec[v] <= desv[v]]

        for v in self.varlist_fec:
            outlier_fec = outlier_fec.loc[outlier_fec[v] <= desv[v]]

        for v in vars:
            outlier = outlier.loc[outlier[v] <= desv[v]]

        outlier_dec = outlier_dec.max().to_dict()
        outlier_fec = outlier_fec.max().to_dict()
        outlier = outlier.max().to_dict()

        # Calcular minimos e máximos
        min_x = data[vars].min().to_dict()
        max_dec = {v: outlier_dec[v] for v in self.varlist_dec}
        max_fec = {v: outlier_fec[v] for v in self.varlist_fec}
        max_x = {v: outlier[v] for v in vars}

        # Criar base ANI
        ani_dec = data.copy()
        ani_fec = data.copy()

        # A análise de sinais é feita pela correlação. Podemos automatizar depois
        for v in self.varlist_dec:
            if self.sinais[v] == '+':
                ani_dec[v] = 100 * (ani_dec[v] - min_x[v]) / (max_dec[v] - min_x[v])
            else:
                ani_dec[v] = 100 - 100 * (ani_dec[v] - min_x[v]) / (max_dec[v] - min_x[v])

        for v in self.varlist_fec:
            if self.sinais[v] == '+':
                ani_fec[v] = 100 * (ani_fec[v] - min_x[v]) / (max_fec[v] - min_x[v])
            else:
                ani_fec[v] = 100 - 100 * (ani_fec[v] - min_x[v]) / (max_fec[v] - min_x[v])

        self.ani_dec = ani_dec[[self.id_var] + self.varlist_dec]
        self.ani_fec = ani_fec[[self.id_var] + self.varlist_fec]

        # Cálculo do Score ANI

        score_ani_dec = []
        score_ani_fec = []
        heterogenio_dec = []
        heterogenio_fec = []

        for i, v in tqdm(ani_dec.iterrows(), total = len(self.ani_dec)):

            elems = self.data_cluster_dec.loc[self.data_cluster_dec[self.id_var] == v[self.id_var]]['Conj_Sem'].to_numpy()

            if len(elems) <= self.minconj and np.max(self.data_cluster_dec.loc[self.data_cluster_dec[self.id_var] == v[self.id_var]]['Heterogeneidade']) > 20:
                mean_ani = ani_dec.loc[ani_dec[self.id_var].isin(elems)][self.varlist_dec].mean().to_dict()
                score_ani_dec.append(np.sum([v[var] - mean_ani[var] for var in self.varlist_dec]) / len(self.varlist_dec))
                heterogenio_dec.append(True)

            else:
                score_ani_dec.append(0)
                heterogenio_dec.append(False)

        for i, v in tqdm(ani_fec.iterrows(), total = len(ani_fec)):

            elems = self.data_cluster_fec.loc[self.data_cluster_fec[self.id_var] == v[self.id_var]]['Conj_Sem'].to_numpy()

            if len(elems) <= self.minconj and np.max(self.data_cluster_fec.loc[self.data_cluster_fec[self.id_var] == v[self.id_var]]['Heterogeneidade']) > 20:
                mean_ani = ani_fec.loc[ani_fec[self.id_var].isin(elems)][self.varlist_fec].mean().to_dict()
                score_ani_fec.append(np.sum([v[var] - mean_ani[var] for var in self.varlist_fec]) / len(self.varlist_fec))
                heterogenio_fec.append(True)
            else:
                score_ani_fec.append(0)
                heterogenio_fec.append(False)

        self.ani_dec['Score ANI DEC'] = score_ani_dec
        self.ani_fec['Score ANI FEC'] = score_ani_fec
        self.ani_dec['Hetero DEC'] = heterogenio_dec
        self.ani_fec['Hetero FEC'] = heterogenio_fec

        ajuste_dec = []
        ajuste_fec = []

        # Empilhar resultados ANI para DEC e FEC

        ani_df = self.ani_dec[[self.id_var] + ['Score ANI DEC', 'Hetero DEC']].merge(self.ani_fec[[self.id_var] + ['Score ANI FEC', 'Hetero FEC']], how = 'inner', on = self.id_var)

        for i, v in ani_df.iterrows():
            if v['Score ANI DEC'] < -3:
                ajuste_dec.append(-0.1)
            elif v['Score ANI DEC'] >= 3 and v['Score ANI DEC'] < 6:
                ajuste_dec.append(0.1)
            elif v['Score ANI DEC'] >= 6 and v['Score ANI DEC'] < 6:
                ajuste_dec.append(0.2)
            elif v['Score ANI DEC'] >= 9 and v['Score ANI DEC'] < 6:
                ajuste_dec.append(0.3)
            else:
                ajuste_dec.append(0)

            if v['Score ANI FEC'] < -3:
                ajuste_fec.append(-0.1)
            elif v['Score ANI FEC'] >= 3 and v['Score ANI FEC'] < 6:
                ajuste_fec.append(0.1)
            elif v['Score ANI FEC'] >= 6  and v['Score ANI FEC'] < 9:
                ajuste_fec.append(0.2)
            elif v['Score ANI FEC'] >= 9:
                ajuste_fec.append(0.3)
            else:
                ajuste_fec.append(0)

        ani_df['Ajuste DEC'] = ajuste_dec
        ani_df['Ajuste FEC'] = ajuste_fec

        self.ani_df = ani_df

Encontrar clusters dinâmicos

In [ ]:
#@title

parametros = {'DesPadMax': 3.0,
              'NumAtributos': 6.0,
              'ContagemMaxima': 200.0,
              'HeteroMax': 20.0,
              'NumConjFixo': 100.0,
              'NumConjMin': 50.0,
              'T': 8.0,
              'Transicao': 5.0,
              'PERAereos': 0.2,
              'MEDAereos': 0.5,
              'PERSubterraneos': 0.5}

parametros = pd.DataFrame([parametros])

preserve_vars = np.unique(list(set(varlist_dec).union(set(varlist_fec))))
dfZ = df[['COD_DIST', 'COD_CONJ', 'PADRAO', 'LOCALIZACAO', dec_y, fec_y] + list(preserve_vars)].dropna().copy()
dfZ.loc[:, dfZ.columns.isin(preserve_vars)] = dfZ.loc[:, dfZ.columns.isin(preserve_vars)].apply(stats.zscore, args = (0, 1))

dfZ_aereo = dfZ.loc[dfZ['PADRAO'].isin([1, 4])]
dfZ_sub = dfZ.loc[dfZ['PADRAO'].isin([2, 3])]

In [ ]:
#@title
sinais_cluster = {v: '-' if pearson.loc[v][dec_y] < 0 else '+' for v in set(varlist_dec).union(set(varlist_fec))}

In [ ]:
benchmark_aereo = 0.2 #@param {type:"slider", min:0, max:1, step:0.01}
benchmark_subterraneo = 0.5 #@param {type:"slider", min:0, max:1, step:0.01}
benchmark_isolado = 0.5 #@param {type:"slider", min:0, max:1, step:0.01}


if caminho == 'Vigente (ANEEL)':

    data_bench = pd.DataFrame({'COD_CONJ': [],
                               'COD_DIST': [],
                               'LOCALIZACAO': [],
                               'PADRAO': [],
                               'NUC': [],
                               'DEC_MEDIO': [],
                               'FEC_MEDIO': [],
                               'DECL_V0': [],
                               'FECL_V0': [],
                               'REGIAO': [],
                               'Score ANI DEC': [],
                               'Hetero DEC': [],
                               'Score ANI FEC': [],
                               'Hetero FEC': [],
                               'Ajuste DEC': [],
                               'Ajuste FEC': [],
                               'Pos DEC': [],
                               'Pos FEC': []
                               })

    # Subterraneo

    print('Subterrâneos')

    # Setup de cluster
    Cluster_Dinamico = Cluster_Dinam(data = dfZ_sub,
                                    varlist_dec = varlist_dec,
                                    varlist_fec = varlist_fec,
                                    id_var = 'COD_CONJ',
                                    sinais = sinais_cluster,
                                    minconj = 50,
                                    maxconj = 100,
                                    maxdp = 3,
                                    maxhetero = 20)

    # Calcular
    Cluster_Dinamico.fit()

    # Correção de heterogeneidade
    Cluster_Dinamico.CalcANI(df)

    data_bench_sub = Cluster_Dinamico.ani_df.merge(df[['COD_CONJ', 'REGIAO', 'COD_DIST', 'LOCALIZACAO', 'PADRAO', 'NUC', dec_y, fec_y, 'DECL_V0', 'FECL_V0']],
                                                  how = 'left', on = 'COD_CONJ', suffixes = (None, '_x'))



    # Encontrar benchmark

    pos_dec = []
    pos_fec = []

    for i, v in data_bench_sub.loc[data_bench_sub['PADRAO'].isin([2,3])].iterrows():
      nconjsem_dec = len(Cluster_Dinamico.data_cluster_dec.loc[Cluster_Dinamico.data_cluster_dec['COD_CONJ'] == v['COD_CONJ']])
      nconjsem_fec = len(Cluster_Dinamico.data_cluster_fec.loc[Cluster_Dinamico.data_cluster_fec['COD_CONJ'] == v['COD_CONJ']])

      try:
        if v['LOCALIZACAO'] == 2:
          percentil = benchmark_isolado
        else:
          if v['PADRAO'] == 1 or v['PADRAO'] == 4:
            percentil = benchmark_aereo
          else:
            percentil = benchmark_subterraneo
      except:
        percentil = benchmark_aereo

      pos_dec.append(int((nconjsem_dec - 1) * (percentil + v['Ajuste DEC']) + 1))
      pos_fec.append(int((nconjsem_fec - 1) * (percentil + v['Ajuste FEC']) + 1))

    data_bench_sub.loc[dfZ_sub.index, 'Pos DEC'] = pos_dec
    data_bench_sub.loc[dfZ_sub.index, 'Pos FEC'] = pos_fec

    # Encontrar alvo regulatório

    decl = []
    fecl = []

    for i, v in data_bench_sub.loc[data_bench_sub['PADRAO'].isin([2,3])].iterrows():
      try:
        decl.append(data_bench_sub.loc[data_bench_sub['COD_CONJ'].isin(Cluster_Dinamico.data_cluster_dec.loc[Cluster_Dinamico.data_cluster_dec['COD_CONJ'] == v['COD_CONJ']]['Conj_Sem'])][dec_y].sort_values().iloc[data_bench_sub[data_bench_sub['COD_CONJ'] == v['COD_CONJ']]['Pos DEC'] - 1].iloc[0])
        fecl.append(data_bench_sub.loc[data_bench_sub['COD_CONJ'].isin(Cluster_Dinamico.data_cluster_fec.loc[Cluster_Dinamico.data_cluster_fec['COD_CONJ'] == v['COD_CONJ']]['Conj_Sem'])][fec_y].sort_values().iloc[data_bench_sub[data_bench_sub['COD_CONJ'] == v['COD_CONJ']]['Pos FEC'] - 1].iloc[0])
      except:
        decl.append(None)
        fecl.append(None)

    data_bench_sub.loc[dfZ_sub.index, 'DECL'] = decl
    data_bench_sub.loc[dfZ_sub.index, 'FECL'] = fecl

    data_bench_sub = data_bench_sub.loc[dfZ_sub.index]


    print('Aéreos')

    # Setup de cluster
    Cluster_Dinamico = Cluster_Dinam(data = dfZ_aereo,
                                    varlist_dec = varlist_dec,
                                    varlist_fec = varlist_fec,
                                    id_var = 'COD_CONJ',
                                    sinais = sinais_cluster,
                                    minconj = 50,
                                    maxconj = 100,
                                    maxdp = 3,
                                    maxhetero = 20)

    # Calcular
    Cluster_Dinamico.fit()

    # Correção de heterogeneidade
    Cluster_Dinamico.CalcANI(df)

    data_bench_aereo = Cluster_Dinamico.ani_df.merge(df[['COD_CONJ', 'REGIAO', 'COD_DIST', 'LOCALIZACAO', 'PADRAO', 'NUC', dec_y, fec_y, 'DECL_V0', 'FECL_V0']],
                                                  how = 'left', on = 'COD_CONJ', suffixes = (None, '_x'))


    # Encontrar benchmark

    pos_dec = []
    pos_fec = []

    for i, v in data_bench_aereo.loc[data_bench_aereo['PADRAO'].isin([1,4])].iterrows():
      nconjsem_dec = len(Cluster_Dinamico.data_cluster_dec.loc[Cluster_Dinamico.data_cluster_dec['COD_CONJ'] == v['COD_CONJ']])
      nconjsem_fec = len(Cluster_Dinamico.data_cluster_fec.loc[Cluster_Dinamico.data_cluster_fec['COD_CONJ'] == v['COD_CONJ']])

      try:
        if v['LOCALIZACAO'] == 2:
          percentil = benchmark_isolado
        else:
          if v['PADRAO'] == 1 or v['PADRAO'] == 4:
            percentil = benchmark_aereo
          else:
            percentil = benchmark_subterraneo
      except:
        percentil = benchmark_aereo

      pos_dec.append(int((nconjsem_dec - 1) * (percentil + v['Ajuste DEC']) + 1))
      pos_fec.append(int((nconjsem_fec - 1) * (percentil + v['Ajuste FEC']) + 1))

    data_bench_aereo.loc[dfZ_aereo.index, 'Pos DEC'] = pos_dec
    data_bench_aereo.loc[dfZ_aereo.index, 'Pos FEC'] = pos_fec

    # Encontrar alvo regulatório

    decl = []
    fecl = []

    for i, v in data_bench_aereo.loc[data_bench_aereo['PADRAO'].isin([1,4])].iterrows():
      try:
        decl.append(data_bench_aereo.loc[data_bench_aereo['COD_CONJ'].isin(Cluster_Dinamico.data_cluster_dec.loc[Cluster_Dinamico.data_cluster_dec['COD_CONJ'] == v['COD_CONJ']]['Conj_Sem'])][dec_y].sort_values().iloc[data_bench_aereo[data_bench_aereo['COD_CONJ'] == v['COD_CONJ']]['Pos DEC'] - 1].iloc[0])
        fecl.append(data_bench_aereo.loc[data_bench_aereo['COD_CONJ'].isin(Cluster_Dinamico.data_cluster_fec.loc[Cluster_Dinamico.data_cluster_fec['COD_CONJ'] == v['COD_CONJ']]['Conj_Sem'])][fec_y].sort_values().iloc[data_bench_aereo[data_bench_aereo['COD_CONJ'] == v['COD_CONJ']]['Pos FEC'] - 1].iloc[0])
      except:
        decl.append(None)
        fecl.append(None)

    data_bench_aereo.loc[dfZ_aereo.index, 'DECL'] = decl
    data_bench_aereo.loc[dfZ_aereo.index, 'FECL'] = fecl

    data_bench_aereo = data_bench_aereo.loc[dfZ_aereo.index]

    data_bench = pd.concat([data_bench, data_bench_aereo, data_bench_sub])


Subterrâneos


Calculando Clusters:   0%|          | 0/23 [00:00<?, ?it/s]

Calculando Clusters:   0%|          | 0/23 [00:00<?, ?it/s]

Consolidando dados de clusters:   0%|          | 0/23 [00:00<?, ?it/s]

  0%|          | 0/3061 [00:00<?, ?it/s]

...

Calculando Clusters:   0%|          | 0/3038 [00:00<?, ?it/s]

Calculando Clusters:   0%|          | 0/3038 [00:00<?, ?it/s]

Consolidando dados de clusters:   0%|          | 0/3038 [00:00<?, ?it/s]

  0%|          | 0/3061 [00:00<?, ?it/s]

  0%|          | 0/3061 [00:00<?, ?it/s]

#### Abordagem alternativa - Fuzzy


Pré-clusterização por Densidade

In [ ]:
#@title Variáveis para pré-clusterização

preclusterizar = True #@param {type:"boolean"}

var_clusters = ['PC_NUC_URB']
var_tratamento = 'Z-Score'

**Pré-Clusterização por Densidade**

**Atenção!
  A checagem de dados deve avaliar se as variáveis inseridas em var_clusters não contém valores missing!**

In [ ]:
#@title
# Tratamento dos dados
if preclusterizar:

  kmeans_data = df[['COD_CONJ', 'PADRAO', 'LOCALIZACAO'] + var_clusters].dropna()

  kmeans_data = kmeans_data.loc[(kmeans_data['PADRAO'] == 1) & kmeans_data['LOCALIZACAO'] == 1]

  kmeans_data[var_clusters] = zscore(kmeans_data[var_clusters], axis = 0)


  # Filtrar os conjuntos apenas para clusterizar os aéreos do SIN
  aereo_sin = kmeans_data

In [ ]:
# Clusterizar:
if preclusterizar:
  min_k = 2 #@param {type:"slider", min:2, max:15, step:1}
  max_k = 15 #@param {type:"slider", min:2, max:20, step:1}

  if preclusterizar:
    df.loc[df.loc[df['LOCALIZACAO'] != 1].index, 'PRECLUSTER'] = 0
    df.loc[df.loc[df['PADRAO'] != 1].index, 'PRECLUSTER'] = 1

    scores = []

    # Nesta etapa vamos escolher qual o melhor número de clusters
    for k in tqdm(range(min_k, max_k + 1)):
      ck = KMeans(k, n_init = 1, max_iter = 999, tol = 1e-5, random_state = 42)
      ck.fit(aereo_sin[var_clusters])
      scores.append(ck.inertia_)

    # Identificar ponto de quebra:
    best_k = KneeLocator(range(min_k, max_k + 1), scores, S=1.0, curve="convex", direction="decreasing").elbow
    print(best_k)


    # Calcular os melhores clusters
    opt_cluster = KMeans(best_k, n_init = 1, max_iter = 999, tol = 1e-5, random_state = 42)
    opt_cluster.fit(aereo_sin[var_clusters])

    df.loc[aereo_sin.index, 'PRECLUSTER'] = opt_cluster.labels_ + min_k

else:
  df['PRECLUSTER'] = None
  df.loc[df.loc[df['LOCALIZACAO'] != 1].index, 'PRECLUSTER'] = 0
  df.loc[df.loc[df['PADRAO'] != 1].index, 'PRECLUSTER'] = 1
  df.loc[df.loc[(df['LOCALIZACAO'] == 1) & (df['PADRAO'] == 1)].index, 'PRECLUSTER'] = 2

  0%|          | 0/14 [00:00<?, ?it/s]

5


In [ ]:
df.groupby('PRECLUSTER').describe()['DEC_MEDIO']

,count,mean,std,min,25%,50%,75%,max
PRECLUSTER,,,,,,,,
0.0,64.0,32.176510,25.836453,3.120000,13.227500,23.248333,43.573333,130.310000
1.0,23.0,1.580870,1.220861,0.013333,0.695833,1.383333,2.180000,4.443333
2.0,667.0,14.690277,10.449022,1.840000,8.228333,11.833333,17.868333,85.126667
3.0,332.0,26.741586,17.738192,5.100000,14.848333,20.846667,32.560833,133.273333
4.0,95.0,38.219667,30.186618,6.733333,14.305000,26.646667,55.355000,128.706667
5.0,566.0,21.474532,13.301668,4.453333,12.577500,18.053333,24.971667,110.566667
6.0,1314.0,8.115041,7.072874,0.183333,4.833333,6.511667,9.108333,159.133333


In [ ]:
df.groupby('PRECLUSTER').describe()['FEC_MEDIO']

,count,mean,std,min,25%,50%,75%,max
PRECLUSTER,,,,,,,,
0.0,64.0,13.568099,7.478477,0.590000,8.882500,12.211667,19.175000,37.080000
1.0,23.0,0.880580,0.839586,0.006667,0.222500,0.753333,1.331667,3.430000
2.0,667.0,6.911404,3.819179,0.986667,4.491667,5.923333,8.180000,28.283333
3.0,332.0,9.976039,6.390176,1.993333,5.458333,7.973333,12.546667,39.830000
4.0,95.0,14.350158,11.366576,2.883333,6.298333,9.616667,19.730000,59.183333
5.0,566.0,8.647400,5.136823,1.800000,5.452500,7.211667,10.091667,48.673333
6.0,1314.0,4.678688,2.780007,0.220000,2.916667,3.968333,5.514583,33.450000


**Clusterização Fuzzy**

In [ ]:
#@markdown Defina o número mínimo e máximo de clusters
min_clusters = 5 #@param {type:"slider", min:1, max:20, step:1}
max_clusters = 10 #@param {type:"slider", min:2, max:20, step:1}

#@markdown Ajuste fino de parâmetros
fuzziness = 1.5 #@param {type:"slider", min:0, max:20, step:0.01}
error = 1e-5 #@param {type:"number"}
maxiter = 999 #@param {type:"number"}

In [ ]:

var_tratamento = 'Min-Max' #@param ['Z-Score', 'Min-Max', 'Linear', 'Log']

# Criar base de dados de clusterização
fuzzy_data = df[['COD_CONJ', dec_y, fec_y, 'PRECLUSTER'] + list(preserve_vars)].dropna()

for v in preserve_vars:
  if var_tratamento == 'Log':
    fuzzy_data[v] = np.log(fuzzy_data[v] + 1)
  elif var_tratamento == 'Z-Score':
    fuzzy_data[v] = zscore(fuzzy_data[v])
  elif var_tratamento == 'Min-Max':
    fuzzy_data[v] = (fuzzy_data[v] - np.min(fuzzy_data[v])) / (np.max(fuzzy_data[v]) - np.min(fuzzy_data[v]))


**Fuzzy Benchmarking**

**DEC**

In [ ]:
#@title
if caminho == 'Fuzzy':

  opt_fk = {}

  for fk in range(int(np.max(fuzzy_data['PRECLUSTER'])) + 1):

    gdata = fuzzy_data[varlist_dec].loc[fuzzy_data['PRECLUSTER'] == fk]

    partition_scores = []

    if fk < 2:
      min_k = 3
      max_k = 6
    else:
      min_k = min_clusters
      max_k = max_clusters

    for k in tqdm(range(min_k, max_k)):
      if len(gdata) > k:
        centers, membership, u0, d, jm, p, fpc = skfuzzy.cmeans(gdata.to_numpy().T, k, fuzziness, error=error, maxiter = maxiter, seed=42)

        gisi = silhouette_score(X=gdata.to_numpy(), labels = np.argmax(membership, axis = 0))

        partition_scores.append(gisi)

      else:
        partition_scores.append(0)

    print(partition_scores)

    opt_fk[fk] = np.argmax(partition_scores) + min_k

  fuzzy_res = {}

  for fk in range(int(np.max(fuzzy_data['PRECLUSTER'])) + 1):

    gdata = fuzzy_data[varlist_dec].loc[fuzzy_data['PRECLUSTER'] == fk]

    centers, membership, u0, d, jm, p, fpc = skfuzzy.cmeans(gdata.to_numpy().T, opt_fk[fk], fuzziness, error=error, maxiter = maxiter, seed=42)

    fuzzy_res[fk] = {'centers': centers,
                    'membership': membership,
                    'd': d}

  dec_ref = {}

  for fk in list(fuzzy_res.keys()):
    avg_fk = []
    for k in range(opt_fk[fk]):
      w = [m if j <= np.quantile(fuzzy_res[fk]['d'][k, :], q = 0.25) else 0 for j, m in zip(fuzzy_res[fk]['d'][k, :], fuzzy_res[fk]['membership'][k, :])]

      if fk >= 2:
        avg_fk.append(quantile(df.loc[df['PRECLUSTER'] == fk]['DEC_MEDIO'], quantile = 0.2, weights = w))
      else:
        avg_fk.append(quantile(df.loc[df['PRECLUSTER'] == fk]['DEC_MEDIO'], quantile = 0.5, weights = w))


    dec_ref[fk] = avg_fk

  dec_ref

  df['DECL'] = None

  for fk in list(fuzzy_res.keys()):
    df.loc[df.loc[df['PRECLUSTER'] == fk].index, 'DECL'] = np.matmul(dec_ref[fk], fuzzy_res[fk]['membership'])

**FEC**

In [ ]:
#@title
if caminho == 'Fuzzy':
  opt_fk = {}

  for fk in range(int(np.max(fuzzy_data['PRECLUSTER'])) + 1):

    gdata = fuzzy_data[varlist_fec].loc[fuzzy_data['PRECLUSTER'] == fk]

    partition_scores = []

    if fk < 2:
      min_k = 3
      max_k = 6
    else:
      min_k = min_clusters
      max_k = max_clusters

    for k in tqdm(range(min_k, max_k)):
      if len(gdata) > k:
        centers, membership, u0, d, jm, p, fpc = skfuzzy.cmeans(gdata.to_numpy().T, k, fuzziness, error=error, maxiter = maxiter, seed=42)

        gisi = silhouette_score(X=gdata.to_numpy(), labels = np.argmax(membership, axis = 0))

        partition_scores.append(gisi)

      else:
        partition_scores.append(0)

    print(partition_scores)

    opt_fk[fk] = np.argmax(partition_scores) + min_k

  opt_fk

  fuzzy_res = {}

  for fk in range(int(np.max(fuzzy_data['PRECLUSTER'])) + 1):

    gdata = fuzzy_data[varlist_fec].loc[fuzzy_data['PRECLUSTER'] == fk]

    centers, membership, u0, d, jm, p, fpc = skfuzzy.cmeans(gdata.to_numpy().T, opt_fk[fk], fuzziness, error=error, maxiter = maxiter, seed=42)

    fuzzy_res[fk] = {'centers': centers,
                    'membership': membership,
                    'd': d}

  fec_ref = {}

  for fk in list(fuzzy_res.keys()):
    avg_fk = []
    for k in range(opt_fk[fk]):
      w = [m if j <= np.quantile(fuzzy_res[fk]['d'][k, :], q = 0.25) else 0 for j, m in zip(fuzzy_res[fk]['d'][k, :], fuzzy_res[fk]['membership'][k, :])]

      if fk >= 2:
        avg_fk.append(quantile(df.loc[df['PRECLUSTER'] == fk]['FEC_MEDIO'], quantile = 0.2, weights = w))
      else:
        avg_fk.append(quantile(df.loc[df['PRECLUSTER'] == fk]['FEC_MEDIO'], quantile = 0.5, weights = w))


    fec_ref[fk] = avg_fk

  df['FECL'] = None

  for fk in list(fuzzy_res.keys()):
    df.loc[df.loc[df['PRECLUSTER'] == fk].index, 'FECL'] = np.matmul(fec_ref[fk], fuzzy_res[fk]['membership'])

In [ ]:
#@title
if caminho == 'Fuzzy':
  data_bench = df[['COD_CONJ', 'REGIAO', 'COD_DIST', 'LOCALIZACAO', 'PADRAO', 'NUC', dec_y, fec_y, 'DECL_V0', 'FECL_V0', 'DECL', 'FECL', 'PRECLUSTER']]

### Definir trajetória

In [ ]:
#@title
# Criar função de trajetória:

def round_up(number, decimal_places):
    factor = 10 ** decimal_places
    return np.ceil(number * factor) / factor

def trajetoria_lim(v0, vlim, horizonte = 8, lim_reduc = 5, formato = 'linear', decimal = 0, trava_v0 = True):

    vlim = round_up(vlim, decimal)

    # Definir trajetória
    traj = []

    for t in range(horizonte + 1):
        if formato == 'linear':
            if v0 >= vlim: # Trava da V0
                if (v0 - t * (v0 - vlim)/horizonte) - int(v0 - t * (v0 - vlim)/horizonte) == 0.5:
                  traj.append(np.round(v0 - t * (v0 - vlim)/horizonte + 0.0001, decimal))
                else:
                  traj.append(np.round(v0 - t * (v0 - vlim)/horizonte, decimal))
            else:
                traj.append(v0)

        elif formato == 'exponencial':
            if v0 >= vlim: # Trava da V0
                traj.append(np.round(v0 * (vlim/v0)^(t / horizonte), decimal))
            else:
                traj.append(v0)

    # Corrigir trajetória pelos limites de redução

    nova_traj = [v0]

    for i in range(1, horizonte + 1):
        if traj[i - 1] - traj[i] > lim_reduc:
            nova_traj.append(nova_traj[i - 1] - lim_reduc)
        else:
            nova_traj.append(traj[i])

    return nova_traj

In [ ]:
#@title
horizonte = 8
limite_dec = 8
limite_fec = 5
formato = 'linear'
var_inicio_dec = 'DECL_V0'
var_inicio_fec = 'FECL_V0'


data_bench[[f'RES_DEC_V{n}' for n in range(11)]] = None
data_bench[[f'RES_FEC_V{n}' for n in range(11)]] = None

#Substituir apenas os limites no horizonte definido
data_bench[[f'RES_DEC_V{n}' for n in range(horizonte + 1)]] = data_bench.apply(lambda v: trajetoria_lim(v[var_inicio_dec], v['DECL'], horizonte, limite_dec), axis = 1, result_type='expand')
data_bench[[f'RES_FEC_V{n}' for n in range(horizonte + 1)]] = data_bench.apply(lambda v: trajetoria_lim(v[var_inicio_fec], v['FECL'], horizonte, limite_fec), axis = 1, result_type='expand')

### Saídas

In [ ]:
#@title
linha = 'ND'
data_bench['LINHA'] = linha
data_bench['MOTOR'] = caminho
data_bench['DATASIM'] = time.strftime("%d%m%Y %H%M")
data_bench['HORIZONTE'] = horizonte

if model_select == 'Pre-selecionada':
  data_bench['CENARIO'] = variaveis_pre
else:
  data_bench['CENARIO'] = model_select

In [ ]:
#@title
# Reordenar:

columns = ['LINHA', 'MOTOR', 'CENARIO', 'DATASIM', 'COD_CONJ',
          'REGIAO', 'COD_DIST', 'LOCALIZACAO', 'PADRAO',
          'NUC', dec_y, fec_y, 'DECL_V0', 'FECL_V0',
          'PRECLUSTER', 'DECL', 'FECL', 'HORIZONTE']

columns = columns + [f'RES_DEC_V{n}' for n in range(11)] + [f'RES_FEC_V{n}' for n in range(11)]

data_bench = data_bench.reindex(columns = columns)

data_bench.to_excel(rf'{output_path}Res_ND_{caminho}_{current_time}_{model_select}.xlsx')